# Introducting  

Bayes theorem is the foundation of machine learning. This kernel is about using this in the classification. Everything you see below is informative and entertaining.  

You will see how a simple non-parametric approach gives a competitive result.  
  
Are your friends lately talking about a priori and a posteriori probabilities? You want to keep the conversation going, but these words are not familiar to you.  
Read this kernel and you will again become the life of the party!

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

In [ ]:
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

Lets get training data from enviroment

In [ ]:
market_train_df, news_train_df = env.get_training_data()

# Summary about data



### Missing values

In [ ]:
import missingno

def plot_miss(df):
    missingno.matrix(df)
    plt.show()
    
plot_miss(market_train_df)

You can see that almost all the observations in the data are filled and we can ignore the missing values.  
  
Many out-of-the-box models suggest that the distribution of features is normal. If we look at the distribution of our signs, we will understand that this approach is applicable in our case as much as the circle is square.

### Numeric features in news data

In [ ]:
def plot_distributions(df, labels, figsize, h, w):
    i=0
    fig = plt.figure(figsize=figsize)
    for l in labels:
        i+=1
        fig.add_subplot(h,w,i)
        plt.tight_layout()
        plt.title(f'{l}, mean = {df[l].mean():.2f}')
        lq = df[l].quantile(0.01)
        uq = df[l].quantile(0.99)
        feature = df[l].dropna()
        plt.hist(feature[(feature > lq)&(feature < uq)], density=True, bins=100, alpha=0.7)
    plt.show()

In [ ]:
news_labels = ['wordCount', 'sentenceCount', 'companyCount', 'bodySize', 'urgency',
              'firstMentionSentence', 'relevance', 'sentimentClass',
              'sentimentNegative', 'sentimentNeutral', 'sentimentPositive',
              'sentimentWordCount', 'noveltyCount12H', 'noveltyCount24H',
              'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H',
              'volumeCounts24H', 'volumeCounts3D', 'volumeCounts5D', 'volumeCounts7D']

plot_distributions(news_train_df[news_labels], news_labels, figsize=(20,25), h=6, w=4)

### Numeric features in price data  


However, market data is normally distributed. If all the data in the world were distributed as well, then data scientists would earn much more.

In [ ]:
market_labels = ['returnsClosePrevMktres10', 'returnsClosePrevMktres1']

plot_distributions(market_train_df[market_labels], market_labels, figsize=(15,5), h=1, w=3)

# State truncating and demension reduction  

To demonstrate the essence of the Bayes equation, we need to translate our continuous features into a limited number of states.  
In market data, the most informative are relative features. This is what we will save, and so that there will not be too much of them, we will save only close features.  
Above, we have seen that some features in the news do not change their value. Let's drop these features and all the rest that contain something other than numbers from 0 to 9.

![](https://i.imgflip.com/2lphog.jpg)

In [ ]:
market_drop_features = ['volume', 'close', 'open', 'returnsClosePrevRaw1', 
    'returnsOpenPrevRaw1','returnsOpenPrevMktres1', 'returnsClosePrevRaw10', 
    'returnsOpenPrevRaw10', 'returnsOpenPrevMktres10']
news_drop_features = ['urgency', 'takeSequence','provider', 'subjects', 
    'audiences','sentimentClass', 'headlineTag', 'sourceTimestamp', 'firstCreated', 
    'sourceId', 'headline', 'marketCommentary', 'assetCodes']

### Clustering sentiment features  
In the title of the competition we are urged to use the news, but all we have just a few semantic features. We will try to explore it in several states through clustering.


In [ ]:
sem_labels = ['sentimentPositive', 'sentimentNegative', 'sentimentNeutral']

In [ ]:
from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(n_clusters = 9, init = 'k-means++')
semant_kmeans = kmeans.fit_predict(news_train_df[sem_labels].values)
semant_kmeans = np.reshape(semant_kmeans, (semant_kmeans.shape[0], 1))

And then we need encode cluster numbers to binary format. I will explain later

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder(categories='auto')
kmean_dummies = onehotencoder.fit_transform(semant_kmeans).toarray()

def dummies_concat(df, dummies, pref):
    dummies = pd.DataFrame(dummies, index=df.index)#dummies[:, 1:]
    dummies.columns = [pref + str(x) for x in dummies.columns]
    return pd.concat([df, dummies], axis=1)

### Numeric features in news data
If you look at the names of the variables in the news, then there is a suspicion that many of them will correlate with each other. Let's look at their correlation.

In [ ]:
def corr_plot(df):
    corr = df.corr()

    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    f, ax = plt.subplots(figsize=(11, 9))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()

In [ ]:
num_labels = ['noveltyCount12H', 'noveltyCount24H','noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D',
             'volumeCounts12H', 'volumeCounts24H', 'volumeCounts3D', 'volumeCounts5D', 'volumeCounts7D',
             'wordCount', 'sentenceCount', 'companyCount', 'bodySize', 'sentimentWordCount', 'relevance',
             'firstMentionSentence']

corr_plot(news_train_df[num_labels])

Indeed, many features tell us about the same thing, and the sign of relevance generally explains all other features. Apparently relevance is some kind of formula that is calculated on the basis of all the features presented. I like the formulas, so I’ll drop everything except this one!
Moreover, I do not think that it is possible to cluster these features. Because of the exponential distributions, we get just one dominant cluster.  
Let's take a closer look at relevance.


In [ ]:
def plot_relevance(s_relevance):
    plt.title(f'Relevance < 1, {sum(news_train_df["relevance"]<1)/news_train_df.shape[0]*100:.2f} % of data')
    plt.hist(s_relevance[s_relevance<1], bins=100)
    plt.show()
    plt.title(f'Relevance = 1, {sum(news_train_df["relevance"] == 1)/news_train_df.shape[0]*100:.2f} % of data')
    plt.hist(s_relevance[s_relevance<1], bins=100)
    plt.hist(s_relevance[s_relevance==1], bins=100)
    plt.show()

In [ ]:
plot_relevance(news_train_df['relevance'])

To translate continuous features into states, create several functions. Relevance can be equal to 1 or not. Returns may be positive or negative. Also, the Gaussian distribution allows us to tell us in which cases the return is abnormal.

In [ ]:
def is_one(x):
    return 1 if x == 1 else 0

def is_positive(x):
    return 1 if x > 0 else 0

def is_abnormal(x, threshold):
    return 1 if np.abs(x) > threshold else 0

# News pooling  
We will not go deep, I will immediately say that for one day in the market we have a few news. In order to take them all, you need to do something like a pooling.
Here, the cluster binarization performed above is useful. For each cluster we will see that it was on that day. And if there are two or more semantic clusters, max pooling will reflect this.

In [ ]:
def max_pooling(df, features, index, agg_method):
    agg_methods = {k: agg_method for k in features}
    return df.groupby(index).agg(agg_methods).reset_index()

## Merging news and price data  
The following function will convert market and news data into a binarized state, such as 0100100010...

In [ ]:
def state_reduce(market, news, kmean_dummies):
    # check positive return
    market['returnsClosePrevMktres1'] = market['returnsClosePrevMktres1'].apply(is_positive)
    market['returnsClosePrevMktres10'] = market['returnsClosePrevMktres10'].apply(is_positive)
    # check abnormal return
    std = market['returnsClosePrevMktres1'].std()
    market['abnormalClosePrevMktres1'] = market['returnsClosePrevMktres1'].apply(lambda x: is_abnormal(x, 2.5*std))
    # check relevance is equal to one
    news['relevance'] = news['relevance'].apply(is_one)
    # drop features, which unusefull for us
    market = market.drop(market_drop_features, axis=1)
    news = news.drop(news_drop_features + sem_labels + num_labels, axis=1)
    # add semant clusters
    news = dummies_concat(news, kmean_dummies, 'sem_cl_')
    # choose features that will be in state
    bin_features = [l for l in news.columns if l not in ['time', 'assetName']]
    # truncat date for merging data
    market['date'] = market['time'].dt.round('d') 
    news['date'] = news['time'].dt.round('d')
    # pooling news in one date
    news = max_pooling(news,  bin_features, ['date', 'assetName'], 'max')
    # merging data and fill NA
    market_train_df = pd.merge(market, news, how='left', on=['date', 'assetName']).fillna(2)
    # reduce state features to str state id
    state_features = bin_features + ['returnsClosePrevMktres1', 'returnsClosePrevMktres10', 'abnormalClosePrevMktres1']
    state = market_train_df[state_features[0]].astype(int).astype(str)
    for col in state_features[1:]:
        state += market_train_df[col].astype(int).astype(str)
    market_train_df['state'] = state
    
    return market_train_df.drop(state_features + ['date', 'assetName'], axis=1)

In [ ]:
%%time

market_train_df = state_reduce(market_train_df, news_train_df, kmean_dummies)

In [ ]:
market_train_df.head()

In [ ]:
print('Count of unique states in train data', len(market_train_df['state'].unique()))

## Bayes  
For each state, it is necessary to obtain the posterior probability of a positive return. To do this, it is necessary to calculate all the components of the Bayes equation. We do not touch the negative return, because Positive and negative returns are mutually exclusive.  
  
$\LARGE P(A \mid B) = \frac{P(B \mid A) \, P(A)}{P(B)}$  

If you multiply these letters into words, you get the following:  
"The probability of event A in the case of observing event B is equal to the product of the probability of event B in case of observing event A and the probability of occurrence of event A divided by the probability of event B"  
For our problem, we can write the equation like this:  
  
$\LARGE P(positiveReturn \mid State_i) = \frac{P(State_i \mid positiveReturn) \, P(positiveReturn)}{P(State_i)}$


In [ ]:
def fit_state_table(df):
    # positive return observations
    pos_reward = df[df['returnsOpenNextMktres10'] > 0]
    # positive return probability
    pos_reward_prob = pos_reward.shape[0] / df.shape[0]
    # probability of each state
    state_probs = train.groupby(['state']).size() / df.shape[0]
    # probability of each state with positive return
    pos_state_probs = pos_reward.groupby(['state']).size() / pos_reward.shape[0]
    # concat all of this to one table
    state_table = pd.concat([state_probs, pos_state_probs], axis = 1, 
                            keys=['state_prob', 'pos_state_probs'], sort=False).fillna(0)
    
    state_table.index.name = 'state'
    
    state_table['pos_return_prob'] = state_table['pos_state_probs'] * pos_reward_prob / state_table['state_prob']
    
    return state_table

# Validation
Now let's see what kind of profit this approach will bring.

In [ ]:
train, test = np.split(market_train_df[['time', 'universe', 'state', 'returnsOpenNextMktres10']], 
                       [int(.8*len(market_train_df))])
print(f'State coverage: { test["state"].isin(train["state"]).sum() / len(test) }')
#test = test[test["state"].isin(train["state"])]

In [ ]:
state_table = fit_state_table(train)
state_table.head()

Let's compare two predictive approaches on test data, and for one we'll see what profit we would get for this period on the exchange. Our confidence will be the rate, the percentage of the starting amount.

In [ ]:
def max_prob_prediction(df, st):
    df = pd.merge(df, st[['pos_return_prob']], how='left', on=['state']).fillna(0)
    return df['pos_return_prob'].apply(lambda p: p if p >= 0.5 else p-1).tolist()

def diff_prob_prediction(df, st):
    df = pd.merge(df, st[['pos_return_prob']], how='left', on=['state']).fillna(0)
    return df['pos_return_prob'].apply(lambda p: 2*p - 1).tolist()

def all_in_prob_prediction(df, st):
    df = pd.merge(df, st[['pos_return_prob']], how='left', on=['state']).fillna(0)
    return df['pos_return_prob'].apply(lambda p: 1 if p >= 0.5 else -1).tolist()

In [ ]:
max_pred = max_prob_prediction(test, state_table)
diff_pred = diff_prob_prediction(test, state_table)
all_in_pred = all_in_prob_prediction(test, state_table)
max_rewards = test['returnsOpenNextMktres10'] * max_pred
diff_rewards = test['returnsOpenNextMktres10'] * diff_pred
all_in_rewards = test['returnsOpenNextMktres10'] * all_in_pred

In [ ]:
def score(r):
    return np.mean(r)/np.std(r)
    
fig = plt.figure(figsize=(14,7))
fig.add_subplot(1,2,1)
plt.tight_layout()
plt.title(f'Rewards')
plt.hist(max_rewards.tolist(), bins=100, alpha=0.5,
         label=f'MAX m={np.mean(max_rewards):.5f}, std={np.std(max_rewards):.5f}')
plt.hist(diff_rewards.tolist(), bins=100, alpha=0.5,
         label=f'DIFF m={np.mean(diff_rewards):.5f}, std={np.std(diff_rewards):.5f}')
plt.hist(all_in_rewards.tolist(), bins=100, alpha=0.5,
         label=f'ALL_IN m={np.mean(all_in_rewards):.5f}, std={np.std(all_in_rewards):.5f}')
plt.legend()
fig.add_subplot(1,2,2)
plt.tight_layout()
plt.title(f'Total reward')
plt.plot(np.cumsum(max_rewards), 
         label=f'MAX sum={np.sum(max_rewards):.2f}, score={score(max_rewards):.4f}')
plt.plot(np.cumsum(diff_rewards), 
         label=f'DIFF sum={np.sum(diff_rewards):.2f}, score={score(diff_rewards):.4f}')
plt.plot(np.cumsum(all_in_rewards), 
         label=f'ALL_IN sum={np.sum(all_in_rewards):.2f}, score={score(all_in_rewards):.4f}')
plt.legend()
plt.show()

If you do not call the broker yet, lets make submission.

In [ ]:
preddays = env.get_prediction_days()
for marketdf, newsdf, predtemplatedf in preddays:
    
    # cpredict semant clusters
    pred_kmeans = kmeans.predict(newsdf[sem_labels].values)
    pred_kmeans = np.reshape(pred_kmeans, (pred_kmeans.shape[0], 1))
    
    # encode semant clusters
    pred_dummies = onehotencoder.transform(pred_kmeans).toarray()
    
    # merge and pool data
    states = state_reduce(marketdf, newsdf, pred_dummies)
    
    # predict confidence
    preds = max_prob_prediction(states, state_table)
    
    #prediction
    predsdf = pd.DataFrame({'ast':states['assetCode'],'conf':preds})
    predtemplatedf.loc[predtemplatedf['assetCode'].isin(predsdf.ast), 'confidenceValue'] = predsdf['conf'].values
    
    env.predict(predtemplatedf)

env.write_submission_file()